# *Drone Delivery: a Vehicle Routing Problem*

Authors: Adrian Menor, Federico Magri, Youssef Farah

// add description 

## Import Packages

In [8]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import gurobipy as gp
from gurobipy import GRB
import timeit
from geopy.distance import geodesic
import pandas as pd
rnd = np.random
rnd.seed(0)

## VRP Model

### Classes 

In [2]:
#DATASET 
#Here we create the dataset
#Initialize
class Drones():
    def __init__(self, name, maxspeed, maxpayload, number_of_drones, power):
        self.name = name
        self.maxspeed = maxspeed
        self.maxpayload = maxpayload
        self.number_of_drones = number_of_drones
        self.power = power

class Clients():
    number_of_clients = 0
    depo_location = (-3.4333, 29.9000)
    def __init__(self,id, number, name, lat, long, demand):
        self.id = id
        self.number = number
        self.name = name
        self.lat = lat
        self.long = long
        self.demand = demand
        Clients.number_of_clients += 1

### Sample Dataset

In [11]:
Clients.number_of_clients = 0
drones = Drones("AAI RQ-7 Shadow", 36.1111, 10, 4, 28.5)#(name, maxspeed, maxpayload, number_of_drones, power consumtion)
infile = open('villages_burundi', 'rb')
list = pickle.load(infile)
boh = pd.read_pickle('villages_burundi')
print(len(list))

client_list = []
for i in range(len(list)):
    client = Clients(list[i][0],i,list[i][1],list[i][2],list[i][3],list[i][4])
    client_list.append(client)
print(len(client_list))
T = 5000 # [s] total delivery duration


1447
1447


In [12]:
print(boh)

[(301396116, 'Muyange-Gasoho', -2.8120115, 30.129159, 73270.71899436777, 2), (498682537, 'Rusengo', -3.3999897, 30.3532833, 50504.67254821779, 2), (672626781, 'Kobero', -2.653605, 30.4069397, 102999.74672198255, 2), (1343330461, 'Gitandu', -3.755371, 29.6677905, 43976.72883792683, 2), (1343357989, 'Butwe', -3.7788147, 29.7229694, 42971.825900084754, 2), (1343387467, 'Mahwa', -3.80367, 29.7819641, 43003.28721471001, 2), (1343387469, 'Kiryama', -3.8671172, 29.722502, 51865.7735876039, 2), (1343387471, 'Kiremba', -3.942023, 29.6747025, 61570.54681730929, 2), (1343414237, 'Vyuya', -3.7059654, 29.6607451, 40195.80738923474, 2), (1343414243, 'Nyakararo', -3.5284079, 29.6044951, 34478.29754267431, 2), (1343414246, 'Ijenda', -3.4803969, 29.5636502, 37735.71339128148, 2), (1344526635, 'Gitaba', -3.873949, 29.9699439, 49342.113076967995, 2), (1344545310, 'Banda', -4.4472927, 29.6634283, 115163.77299536394, 2), (1344545313, 'Gatete', -4.046748, 29.4784727, 82426.14453867933, 2), (1344545314, 'Kar

In [6]:
print(list[:3])

[(301396116, 'Muyange-Gasoho', -2.8120115, 30.129159, 73270.71899436777, 2), (498682537, 'Rusengo', -3.3999897, 30.3532833, 50504.67254821779, 2), (672626781, 'Kobero', -2.653605, 30.4069397, 102999.74672198255, 2)]


In [ ]:
lat= [xc.lat for xc in client_list] # customer x locations
long = [yc.long for yc in client_list] #rnd.rand(n-1)*100 # customer y locations
depo_lat = Clients.depo_location[0]; depo_long = Clients.depo_location[1]
dist_from_depo = {(i): geodesic((Clients.depo_location[0],Clients.depo_location[1]),(lat[i],long[i])) for i in range(len(lat))} # euclidean distances TODO change km to m








In [ ]:
#Plotting
BBox = ((min(long)-0.1,   max(long)+0.1,  min(lat)-0.1, max(lat)+0.1))
ruh_m = plt.imread('map.png')

fig, ax = plt.subplots(figsize = (8,7))


ax.plot(depo_long, depo_lat, c='r', marker='s')
ax.scatter(long, lat, c='b')
ax.set_xlim(BBox[0],BBox[1])
ax.set_ylim(BBox[2],BBox[3])


ax.imshow(ruh_m, zorder=0, extent = BBox, aspect= 'equal')
plt.show()

## Model Setup

In [ ]:
# Basic problem variables
n = Clients.number_of_clients # nodes
clients = [i for i in range(1,n+1)]   # N_0
nodes = [0]+clients                   # N
N_N_0 = [(i,j) for i in nodes for j in clients if i!=j]
lat= [Clients.depo_location[0]]+[xc.lat for xc in client_list] # customer x locations
long = [Clients.depo_location[1]]+[yc.long for yc in client_list] #rnd.rand(n-1)*100 # customer y locations



In [ ]:
#Plotting
BBox = ((min(long)-0.1,   max(long)+0.1,  min(lat)-0.1, max(lat)+0.1))
ruh_m = plt.imread('map.png')

for i, j in active_arcs:
    ax.plot([long[i], long[j]], [lat[i], lat[j]], c='g', linestyle= ':', zorder=1)
    ax.annotate(nodes[i], (long[i]+0.01, lat[i]+0.01))

ax.plot(long[0], lat[0], c='r', marker='s')
ax.scatter(long[1:], lat[1:], c='b')
ax.set_xlim(BBox[0],BBox[1])
ax.set_ylim(BBox[2],BBox[3])


ax.imshow(ruh_m, zorder=0, extent = BBox, aspect= 'equal')
plt.show()

In [ ]:
# Drone parameters
M = drones.number_of_drones # Number of drones
K = 1000000                 # upper bound payload weight [kg]
v = drones.maxspeed         # drone speed                [m/s]
Q = drones.maxpayload       # max drone payload [kg] 
p = drones.power            #Power Consumption [kW]

In [ ]:
# Decision variables
arcs = [(i,j) for i in nodes for j in nodes if i!=j]      # fully connected links (N x N)
sigma_var = [(i,j) for i in clients for j in clients ]    # going through depot
y = arcs # payload weight between paths
t = [i for i in nodes] # time at node i
a = [i for i in clients] # time between node i and depot
z = a # The energy consumed from a drone’s battery by the time it arrives at the depot directly after leaving
f = t #Enegry cosumed at location i

In [ ]:
# Costs
s = {(i, j): geodesic((lat[i],long[i]),(lat[j],long[j])).m for i, j in arcs} # euclidean distances TODO change km to m
D = {i.number: i.demand for i in client_list}# demand of client rnd.randint(1,5)


In [ ]:
### Creating the Model ###
m = gp.Model('CVRP')

In [ ]:
# Adding decision variables
x = m.addVars(arcs,vtype = GRB.BINARY,name='x') # x = arcs 
sigma = m.addVars(sigma_var,vtype = GRB.BINARY,name='sigma') 
y = m.addVars(y,vtype = GRB.CONTINUOUS,name='y') # payload weight between paths
t = m.addVars(t,vtype = GRB.CONTINUOUS,name='t') # time at node i
a = m.addVars(a,vtype = GRB.CONTINUOUS,name='a') # time between node i and the depot 
f = m.addVars(f,vtype = GRB.CONTINUOUS,name='f')
z = m.addVars(z,vtype = GRB.CONTINUOUS,name='z')
m.update()

In [ ]:
# Objective function
m.setObjective(gp.quicksum(s[i,j]*x[i,j] for i,j in arcs),GRB.MINIMIZE)

In [ ]:
# Constraints
m.addConstrs((gp.quicksum(x[i,j] for j in nodes if j!= i) == 1 for i in clients),name = "4a") # (4a) each node visited exactly once by a drone

m.addConstrs((gp.quicksum(x[i,j] for j in nodes if j!= i)-gp.quicksum(x[j,i] for j in nodes if j!= i)== 0 for i in nodes),name = "4b") # (4b) when node i is visited, the drone leaves

# Reusability Constraints
m.addConstrs((gp.quicksum(sigma[i,j] for j in clients) <= x[i,0] for i in clients),name = "5a") # (5a) if drone comes back to depot from location i, it can fly again
m.addConstrs((gp.quicksum(sigma[j,i] for j in clients) <= x[0,i] for i in clients),name = "5b") # (5b) if drone goes from depot to location i, it arrived previously from somewhere else
m.addConstr((gp.quicksum(x[0,i] for i in clients) - gp.quicksum(sigma[i,j] for i,j in sigma_var if i!=j) <= M),name = "5c") # (5c) more than M drones cannot fly simultaneously

# Demand Constraints
m.addConstrs((gp.quicksum(y[j,i] for j in nodes if j!=i) - gp.quicksum(y[i,j] for j in nodes if j!=i) == D[i] for i in clients),name = "6a") # (6a) payload when leaving i less than arriving i with Di kg less
m.addConstrs((y[i,j] <= K*x[i,j] for i,j in arcs if i!=j),name = "6b") # (6b) sets the payload weight of each edge without a vehicle to 0 kg

# Time Constraints
m.addConstrs((t[i] - t[j] + s[i,j]/v <= K* (1-x[i,j]) for i,j in N_N_0 if i!=j),name = "7a") # (7a) keeps track of time i that each location i is visited by a drone
m.addConstrs((t[i] - a[i] + s[i,0]/v <= K * (1 - x[i,0]) for i in clients),name = "7b") # (7b) keeps track of the time ai that a drone arrives at the depot from location i 
m.addConstrs((a[i] - t[j] + s[0,j]/v <= K * (1 - sigma[i,j]) for i,j in sigma_var if i!=j),name = "7c") # (7c) ensures that times are correct for drones that are reused after returning to the depot
m.addConstrs((t[i] <= T  for i in clients),name = "7d_7e") # (7d)  and (7e) checks delivery time 

# Capacity Constraints
m.addConstrs((y[i,j] <= Q * x[i,j] for i,j in arcs if i!=j), name = "8a") #(8a) capacity of drone between i and j is restricted by Q

# Energy Constraints
m.addConstrs((f[i] - f[j] + p*s[i,j]/v <= K*(1-x[i,j]) for i,j in N_N_0 if i!=j), name = "9a")#(9a) forces fi to equal the total energy consumed along the route up to location i.
m.addConstrs((f[i] - z[i] + p*s[i,0]/v <= K * (1 - x[i,0]) for i in clients),name = "9b")#(9b) makes zi equal to the energy consumedflying the entire route that ends at location i
m.addConstrs((z[i]<= K*x[i,0] for i in clients),name = "9c") #9c

In [ ]:
m.update()
#Writing LP file
#m.Params.timeLimit = 200 #[s]
m.Params.MIPGap = 0
m.optimize()
m.write('model_lab.lp')

In [ ]:
def loop_finder(arc):
    starting_list = []
    for i in range(0,len(arc)):
        if arc[i][0] == 0:
            starting_list.append(arc[i])
    sorted_list = []
    for i in range(0, len(starting_list)):
        sorted_list.append(starting_list[i])
        j = 0
        while j < len(arc):
            j += 1
            if sorted_list[-1][1]== arc[j][0]:
                sorted_list.append(arc[j])
                j = 0 
            if sorted_list[-1][1]== 0:
                j = len(arc)
    index = []
    for i in range(0,len(sorted_list)):
        if sorted_list[i][0]== 0:
            index.append(i)
    print(len(index))
    index.append(len(sorted_list))
    loops = []
    for i in range(0,len(index)):
        loops.append(sorted_list[index[i-1]:index[i]])
    loops = loops[1:]
    return loops

#Plotting

BBox = ((min(long)-0.1,   max(long)+0.1,  min(lat)-0.1, max(lat)+0.1))
ruh_m = plt.imread('map.png')

active_arcs = [a for a in arcs if x[a].x > 0.99]
sorted_arcs = loop_finder(active_arcs)
fig, ax = plt.subplots(figsize = (8,7))
color = ['g','r','b',"y"]
linestyle = [':' ]
for k in range(len(sorted_arcs)):
    for i, j in sorted_arcs[k]:
        ax.plot([long[i], long[j]], [lat[i], lat[j]], c=color[k], linestyle= ':', zorder=1)
        ax.annotate(nodes[i], (long[i]+0.01, lat[i]+0.01))
ax.plot(long[0], lat[0], c='r', marker='s')
ax.scatter(long[1:], lat[1:], c='b')
ax.set_xlim(BBox[0],BBox[1])
ax.set_ylim(BBox[2],BBox[3])


'''
#Plotting
BBox = ((min(long)-0.1,   max(long)+0.1,  min(lat)-0.1, max(lat)+0.1))
ruh_m = plt.imread('map.png')

active_arcs = [a for a in arcs if x[a].x > 0.99]
fig, ax = plt.subplots(figsize = (8,7))

for i, j in active_arcs:
    ax.plot([long[i], long[j]], [lat[i], lat[j]], c='g', linestyle= ':', zorder=1)
    ax.annotate(nodes[i], (long[i]+0.01, lat[i]+0.01))

ax.plot(long[0], lat[0], c='r', marker='s')
ax.scatter(long[1:], lat[1:], c='b')
ax.set_xlim(BBox[0],BBox[1])
ax.set_ylim(BBox[2],BBox[3])


ax.imshow(ruh_m, zorder=0, extent = BBox, aspect= 'equal')
plt.show()'''

In [ ]:
for v in m.getVars():
        print(str(v.VarName)+'='+str(v.x))
print(m.objVal)